## 닮은꼴 분석 앱을 만들어보자.
- 1. 드라이브 마운트 -> 딥러닝 작업폴더 이동
- 2. 데이터 압출풀기
- 3. 데이터로딩 with image_dataset_from_directory
- 4. 훈련/평가 데이터분리 -> 데이터셋이 적으니 9:1 or 8:2
- 5. 이미지 특징추출 모델 다운로드 with inceptionV3
- 6. 인공신경만 설계 및 학습 -> 뉴련의 마지막 층은 범주 갯수로 맞출것
- 7. 조기학습중단, 모델자동저장 생성
- 8. 학습 및 평가 -> 검증데이터 설정
- 9. 앱 만들기 -> gradio 활용

In [4]:
import numpy as np
import pandas as pd

from tqdm import tqdm # 반복문 작업의 진척률을 알려주는 도구
from tensorflow.keras.utils import image_dataset_from_directory # 폴더로부터 이미지를 로딩하는 함수
from tensorflow.keras.applications import InceptionV3 # 이미지 특징을 추출하는 도구
from tensorflow.keras.applications.inception_v3 import preprocess_input # 이미지의 픽셀값을 0~255 -> 1 ~ -1 로변환
from tensorflow.keras.models import Sequential # 신경망을 하나로 묶어주는툴
from tensorflow.keras.layers import Dense # 뉴런의 묶음
# 분류모델 평가지표 확인
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
# 학습 조기 중단 -> 의미없이 epoch가 계속 반복하는 것을 방지하고 지정된 횟수보다 일찍 멈추는것
from tensorflow.keras.callbacks import EarlyStopping
# 모델 자동 저장 -> 학습 도중에 중간중간 저장하는 기능
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.utils import validation
from tensorflow.keras.models import load_model

In [5]:
!pwd

/content


In [6]:
%cd "/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning"

/content/drive/MyDrive/Colab Notebooks/Ai_Deep_Learning


In [7]:
# 압축풀기
# !unzip ./data/img_idol.zip -d ./data/img_idol/

In [8]:
train_data = image_dataset_from_directory(
    directory = "./data/img_idol/이미지(슈화,김다미)", # 읽어들일 폴더
    labels = "inferred", # 폴더명을 인식해서 사진마다 정답으로 붙여 줌
    label_mode = 'categorical', # 다중분류 형태로 정답이 생성
    color_mode = 'rgb', # 컬러사진 설정
    image_size = (224 ,224) # 읽어들인 사진의 크기 crop

)

Found 706 files belonging to 2 classes.


In [9]:
X_train = [] # 읽어들인 사진데이터가 저장될 리스트
y_train = [] # 읽어들인 정답데이터가 저장될 리스트

In [10]:
# 32장씩 문제와 정답 로딩
for img, label in tqdm(train_data.as_numpy_iterator()): #tqdm -> 반복된 횟수를 알수있다.
  X_train.append(img)
  y_train.append(label)

23it [01:34,  4.11s/it]


In [11]:
# 리스트에 담겨있는 사진데이터 및 정답데이터를 하나의 넘파이로 통합
X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)
print(X_train.shape, y_train.shape)

(706, 224, 224, 3) (706, 2)


In [12]:
X_train = preprocess_input(X_train)

In [13]:
# 이미지 특지 추출기 생성
imageEmbedding = InceptionV3(include_top=False,
                             weights = 'imagenet',
                             input_shape=(224,224,3),
                             pooling='avg'
                            )

87910968/87910968 [==============================] - 3s 0us/step


In [14]:
my_idol_model = Sequential() #틀 생성
my_idol_model.add(imageEmbedding)
my_idol_model.add(Dense(units=100, activation='relu'))
my_idol_model.add(Dense(units=5, activation='relu'))
my_idol_model.add(Dense(units=2, activation='softmax'))

In [15]:
my_idol_model.compile(loss='categorical_crossentropy', # 모델의 잘못된 정도를 파악하는 알고리즘
                        optimizer='Adam', # 모델의 최적화를 도와주는 알고리즘
                        metrics=['accuracy'])

In [16]:
# 조기학습 중단
early = EarlyStopping(monitor='val_accuracy', patience=5) # 자동중단을 위해 지켜볼 모델의 성능지표

# 모델자동저장
# model_path = "./data/animal-{epoch:02d}-{val_accuracy:.2f}.hdf5"
model_path = "./data/idol-{epoch:02d}-{val_accuracy:.2f}.hdf5"
mdkpt = ModelCheckpoint(filepath=model_path,
                        save_best_only = True,
                        monitor='val_accuracy'
                        )

In [17]:
my_idol_model.fit(X_train, y_train, epochs=10)

Epoch 1/10
23/23 [==============================] - 64s 314ms/step - loss: 0.5214 - accuracy: 0.7734
Epoch 2/10
23/23 [==============================] - 5s 220ms/step - loss: 0.3689 - accuracy: 0.8683
Epoch 3/10
23/23 [==============================] - 5s 224ms/step - loss: 0.2370 - accuracy: 0.9079
Epoch 4/10
23/23 [==============================] - 5s 227ms/step - loss: 0.3312 - accuracy: 0.8541
Epoch 5/10
23/23 [==============================] - 5s 225ms/step - loss: 0.2149 - accuracy: 0.9278
Epoch 6/10
23/23 [==============================] - 5s 230ms/step - loss: 0.1654 - accuracy: 0.9292
Epoch 7/10
23/23 [==============================] - 5s 231ms/step - loss: 0.0772 - accuracy: 0.9745
Epoch 8/10
23/23 [==============================] - 5s 231ms/step - loss: 0.2506 - accuracy: 0.9221
Epoch 9/10
23/23 [==============================] - 5s 238ms/step - loss: 0.1147 - accuracy: 0.9575
Epoch 10/10
23/23 [==============================] - 5s 236ms/step - loss: 0.2909 - accuracy: 0.902

In [19]:
# 저장된 모델 불러오기
best_model= load_model('./data/idol-09-0.94.hdf5')

In [ ]:
!pip install gradio

In [24]:
import gradio as gr
#예측 알고리즘 함수
def my_predict(input_img) :
  input_img = input_img.reshape(1,224,224,3) #1장의 사진을 4차원으로 만들어줌
  # input_img = preprocess_input(input_img)
  pre = best_model.predict(input_img) # 모델예측
  class_names = ['김다미','슈화']
  return class_names[pre.argmax(1)[0]]

demo = gr.Interface(my_predict, #연결된 함수의 return 값을 화면에 연결
                    inputs=gr.Image(shape=(224,224)), #생성될 입력 인터페이스 설정
                    outputs="text") #생성될 출력 인터페이스 설정
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8bfb9b98c7404adf28.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [25]:
best_model.evaluate(X_train, y_train)

23/23 [==============================] - 4s 76ms/step - loss: 0.7037 - accuracy: 0.4023


[0.7037235498428345, 0.4022662937641144]

In [ ]:
# print(X_train.max(), X_train.min())

In [ ]:
# print(X_train.max(), X_train.min())

In [ ]:
# imageEmbedding = InceptionV3(include_top=False,
#                              weights ='imagenet',
#                              input_shape=(224,224,3),
#                              pooling='avg')

In [ ]:
# my_idol_model = Sequential() # 틀 생성
# my_idol_model.add(imageEmbedding)
# my_idol_model.add(Dense(units=100, activation='relu'))
# my_idol_model.add(Dense(units=5, activation='relu'))
# my_idol_model.add(Dense(units=2, activation='softmax'))

In [ ]:
# my_idol_model.compile(loss="categorical_crossentropy",  # 모델의 잘못된 정도를 파악하는 알고리즘
#                         optimizer="Adam", # 모델의 최적화를 도와주는 알고리즘
#                         metrics=["accuracy"])

In [ ]:
# my_idol_model.fit(X_train, y_train, epochs=1)

In [ ]:
# # 조기학습 중단
# early = EarlyStopping(monitor='val_accuracy', patience=5) # 자동중단을 위해 지켜볼 모델의 성능지표

# # 모델자동저장
# # model_path = "./data/animal-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# model_path = "./data/idol-{epoch:02d}-{val_accuracy:.2f}.hdf5"
# mdkpt = ModelCheckpoint(filepath=model_path,
#                         save_best_only = True,
#                         monitor='val_accuracy'
#                         )

In [ ]:

# my_idol_model.fit(X_train, y_train,
#                     validation_split= 0.2,
#                     epochs=10,\
#                     callbacks=[early,mdkpt]
#                     )

In [ ]:
# best_model = load_model("")

In [ ]:
# best_model.evaluate(X_test, y_test)

In [ ]:
# test 아직 설정전
